<a href="https://colab.research.google.com/github/akatjizeu/Data-Science/blob/main/Random_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [ ]:
url = '/content/credit.csv'
df = pd.read_csv(url)
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes


In [ ]:
df.shape

(1000, 17)

In [ ]:
df['default'].value_counts()

,count
default,
no,700
yes,300


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
months_loan_duration,1000.0,20.903,12.058814,4.0,12.0,18.0,24.00,72.0
amount,1000.0,3271.258,2822.736876,250.0,1365.5,2319.5,3972.25,18424.0
percent_of_income,1000.0,2.973,1.118715,1.0,2.0,3.0,4.00,4.0
years_at_residence,1000.0,2.845,1.103718,1.0,2.0,3.0,4.00,4.0
age,1000.0,35.546,11.375469,19.0,27.0,33.0,42.00,75.0
existing_loans_count,1000.0,1.407,0.577654,1.0,1.0,1.0,2.00,4.0
dependents,1000.0,1.155,0.362086,1.0,1.0,1.0,1.00,2.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   checking_balance      1000 non-null   object
 1   months_loan_duration  1000 non-null   int64 
 2   credit_history        1000 non-null   object
 3   purpose               1000 non-null   object
 4   amount                1000 non-null   int64 
 5   savings_balance       1000 non-null   object
 6   employment_duration   1000 non-null   object
 7   percent_of_income     1000 non-null   int64 
 8   years_at_residence    1000 non-null   int64 
 9   age                   1000 non-null   int64 
 10  other_credit          1000 non-null   object
 11  housing               1000 non-null   object
 12  existing_loans_count  1000 non-null   int64 
 13  job                   1000 non-null   object
 14  dependents            1000 non-null   int64 
 15  phone                 1000 non-null   o

In [ ]:
#Convert columns with 'object' data type to categorical variables
for feature in df.columns: #loop through all columns in te dataframe
    if df[feature].dtype == 'object': #only apply for columns with categorical strings
        df[feature] = pd.Categorical(df[feature])#.codes   # Replace strings with an integer
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,other_credit,housing,existing_loans_count,job,dependents,phone,default
0,< 0 DM,6,critical,furniture/appliances,1169,unknown,> 7 years,4,4,67,none,own,2,skilled,1,yes,no
1,1 - 200 DM,48,good,furniture/appliances,5951,< 100 DM,1 - 4 years,2,2,22,none,own,1,skilled,1,no,yes
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 years,2,3,49,none,own,1,unskilled,2,no,no
3,< 0 DM,42,good,furniture/appliances,7882,< 100 DM,4 - 7 years,2,4,45,none,other,1,skilled,2,no,no
4,< 0 DM,24,poor,car,4870,< 100 DM,1 - 4 years,3,4,53,none,other,2,skilled,2,no,yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   checking_balance      1000 non-null   category
 1   months_loan_duration  1000 non-null   int64   
 2   credit_history        1000 non-null   category
 3   purpose               1000 non-null   category
 4   amount                1000 non-null   int64   
 5   savings_balance       1000 non-null   category
 6   employment_duration   1000 non-null   category
 7   percent_of_income     1000 non-null   int64   
 8   years_at_residence    1000 non-null   int64   
 9   age                   1000 non-null   int64   
 10  other_credit          1000 non-null   category
 11  housing               1000 non-null   category
 12  existing_loans_count  1000 non-null   int64   
 13  job                   1000 non-null   category
 14  dependents            1000 non-null   int64   
 15  phone

In [ ]:
#Print value counts for all columns of data type cetogorical
for feature in df.columns:
    if df[feature].dtype == 'category':
        print(df[feature].value_counts())

checking_balance
unknown       394
< 0 DM        274
1 - 200 DM    269
> 200 DM       63
Name: count, dtype: int64
credit_history
good         530
critical     293
poor          88
very good     49
perfect       40
Name: count, dtype: int64
purpose
furniture/appliances    473
car                     337
business                 97
education                59
renovations              22
car0                     12
Name: count, dtype: int64
savings_balance
< 100 DM         603
unknown          183
100 - 500 DM     103
500 - 1000 DM     63
> 1000 DM         48
Name: count, dtype: int64
employment_duration
1 - 4 years    339
> 7 years      253
4 - 7 years    174
< 1 year       172
unemployed      62
Name: count, dtype: int64
other_credit
none     814
bank     139
store     47
Name: count, dtype: int64
housing
own      713
rent     179
other    108
Name: count, dtype: int64
job
skilled       630
unskilled     200
management    148
unemployed     22
Name: count, dtype: int64
phone
no     596

In [ ]:
#Create a dictionary to use to replace all category viariables with digital values
replaceStruct = {
    'checking_balance': {'< 0 DM':1,'1 - 200 DM':2,'> 200 DM':3,'unknown':4},
    'credit_history': {'critical':1,'poor':2,'good':3,'very good':4,'perfect':5},
    'savings_balance': {'< 100 DM':1,'100 -500 DM':2,'500 -1000 DM':3,'1000 -2000 DM':4,'> 2000 DM':5,'unknown':6},
    'employment_duration': {'< 1 year':1,'1 - 4 years':2,'4 - 7 years':3,'7 -10 years':4,'> 10 years':5,'unemployed':6},
    'phone': {'no':1,'yes':2},
    'default': {'no':0,'yes':1}
}

oneHotColumns = ['purpose', 'housing', 'other_credit','job']

In [ ]:
#Do one hot encoding and label encoding
df = df.replace(replaceStruct)
df = pd.get_dummies(df,columns=oneHotColumns)
df.head()

,checking_balance,months_loan_duration,credit_history,amount,savings_balance,employment_duration,percent_of_income,years_at_residence,age,existing_loans_count,...,housing_other,housing_own,housing_rent,other_credit_bank,other_credit_none,other_credit_store,job_management,job_skilled,job_unemployed,job_unskilled
0,1,6,1,1169,6,> 7 years,4,4,67,2,...,False,True,False,False,True,False,False,True,False,False
1,2,48,3,5951,1,2,2,2,22,1,...,False,True,False,False,True,False,False,True,False,False
2,4,12,1,2096,1,3,2,3,49,1,...,False,True,False,False,True,False,False,False,False,True
3,1,42,3,7882,1,3,2,4,45,1,...,True,False,False,False,True,False,False,True,False,False
4,1,24,2,4870,1,2,3,4,53,2,...,True,False,False,False,True,False,False,True,False,False


In [ ]:
#Split the data into train and test sets
X = df.drop('default',axis=1)
y = df['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
#Scale the data
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test

In [ ]:
#Fucntions to calculate different metrics
def confucsion_matrix(model, y_actual, label=[1,0]):
  '''
  model: classifier to predict values of X
  y_actual: correct labels
  label: list of labels, same order as y_actual
  '''
  y_predict = model.predict(X_test)
  cm = metrics.confusion_matrix(y_actual, y_predict, labels=label)
  df_cm = pd.DataFrame(cm, index= [i for i in ["Actual - No", "Actual - Yes"]], columns = [i for i in ["Predicted - No", "Predicted - Yes"]])
  group_counts = ["{0:0.0f}".format(value) for value in cm.flatten()]
  group_percentages = ["{0:.2%}".format(value) for value in cm.flatten()/np.sum(cm)]
  labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_counts,group_percentages)]
  labels = np.asarray(labels).reshape(2,2)
  plt.figure(figsize=(10,7))
  sns.heatmap(df_cm, annot=labels, fmt='', cmap='Blues')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

In [ ]:
#FUnction to calculate different mteric scores of the model - Accuracy, Recall and Precision
def model_scores(model, flag=True):
    '''
    model: classifier to predict values of X
    X_test: explanatory variables
    y_test: correct labels
    '''
